In [5]:
import rospy
import yaml
from visualization_msgs.msg import Marker, MarkerArray
from geometry_msgs.msg import Point, Pose, Quaternion
import os

# Inicializar ROS (Solo si no está ya inicializado)
if not rospy.core.is_initialized():
    rospy.init_node("trajectory_visualization", anonymous=True)

# Publicador de marcadores en Rviz
marker_pub = rospy.Publisher("/visualization_marker_array", MarkerArray, queue_size=10)


In [1]:
# Ruta del archivo YAML
yaml_file_path = "/mnt/data/toolpath.yaml"  # Asegúrate de que este archivo existe

# Verificar si el archivo existe antes de cargarlo
if not os.path.exists(yaml_file_path):
    rospy.logerr(f"File {yaml_file_path} not found! Make sure it's in the correct location.")
    exit()

# Cargar el YAML
with open(yaml_file_path, "r") as file:
    toolpath_data = yaml.safe_load(file)

# Convertir los datos del YAML en una lista de Poses de ROS
toolpath_poses = []
for pose_data in toolpath_data["poses"]:
    pose = Pose(
        position=Point(
            pose_data["position"]["x"],
            pose_data["position"]["y"],
            pose_data["position"]["z"]
        ),
        orientation=Quaternion(
            pose_data["orientation"]["x"],
            pose_data["orientation"]["y"],
            pose_data["orientation"]["z"],
            pose_data["orientation"]["w"]
        )
    )
    toolpath_poses.append(pose)

rospy.loginfo(f"Loaded {len(toolpath_poses)} poses from YAML file.")


In [2]:
# Crear MarkerArray para la trayectoria
marker_array = MarkerArray()

for i, pose in enumerate(toolpath_poses):
    marker = Marker()
    marker.header.frame_id = "base_link"
    marker.header.stamp = rospy.Time.now()
    marker.ns = "trajectory"
    marker.id = i
    marker.type = Marker.SPHERE
    marker.action = Marker.ADD
    marker.pose.position = pose.position
    marker.scale.x = 0.02
    marker.scale.y = 0.02
    marker.scale.z = 0.02
    marker.color.r = 1.0
    marker.color.g = 0.0
    marker.color.b = 0.0
    marker.color.a = 1.0  # Opacidad
    marker.lifetime = rospy.Duration()
    marker_array.markers.append(marker)

# Publicar los marcadores en Rviz
rospy.sleep(1)  # Esperar a que Rviz se conecte
marker_pub.publish(marker_array)
rospy.loginfo("Trajectory published to Rviz!")

# Mantener el nodo activo para visualizar en Rviz
rospy.spin()


Loaded toolpath from CSV.


In [ ]:
poses_from_yaml()

FileNotFoundError: [Errno 2] No such file or directory: '/mnt/data/toolpath_spiral.csv'

In [60]:
execute_trajectory_srv()


success: False

In [ ]:
if CAPTURE:
    start_recon_req, stop_recon_req = gen_recon_msg("/home/huanyu/capture")

resp = plan_goal_srv(Goal(pose=trajectory_poses[0], vel_scale=0.1, acc_scale=0.1, planner="ptp"))
if resp.success and not resp.configuration_change:
    print("Start recon")
    if CAPTURE:
        start_recon(start_recon_req)
    execute_trajectory_srv()
else:
    rospy.loginfo("Failed to plan")
    exit()

resp = plan_goal_srv(Goal(pose=trajectory_poses[-1], vel_scale=0.1, acc_scale=0.1, planner="lin"))
if resp.success and not resp.configuration_change:
    execute_trajectory_srv()
else:
    rospy.loginfo("Failed to plan")
    exit()

if CAPTURE:
    stop_recon(stop_recon_req)


NameError: name 'gen_recon_msg' is not defined